In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_16040\549052057.py:1: DtypeWarning: Columns (13,36,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./서울시 휴게음식점 인허가 정보.csv', encoding='cp949')
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_16040\549052057.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  gg_df = pd.read_csv('./경기도 휴게음식점현황.csv', encoding='cp949')


In [17]:
df.keys()

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '위생업태명', '남성종사자수', '여성종사자수',
       '영업장주변구분명', '등급구분명', '급수시설구분명', '총인원', '본사종업원수', '공장사무직종업원수',
       '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모',
       '전통업소지정번호', '전통업소주된음식', '홈페이지'],
      dtype='object')

In [18]:
gg_df.keys()

Index(['시군명', '사업장명', '인허가일자', '소재지도로명주소', '소재지지번주소', '소재지우편번호', '위도', '경도',
       '인허가취소일자', '통합영업상태명', '영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자',
       '소재지시설전화번호', '소재지면적정보', '업태구분명정보', 'X좌표값', 'Y좌표값', '위생업태명', '남성종사자수',
       '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총종업원수', '본사종업원수',
       '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액', '월세액',
       '다중이용업소여부', '시설총규모정보', '전통업소지정번호', '전통업소주된음식', '홈페이지URL'],
      dtype='object')

In [19]:
gg_sb_df = gg_df[['사업장명','통합영업상태명', '인허가일자', '폐업일자', 'X좌표값', 'Y좌표값']]
gg_sb_df = gg_sb_df[gg_sb_df['사업장명'].str.contains('스타벅스')]
gg_sb_df.reset_index(drop=True, inplace=True)
gg_sb_df = gg_sb_df.rename(columns={'통합영업상태명' : '영업상태', '인허가일자' : '개업일자'})

In [20]:
seoul_sb_df = df[['사업장명','상세영업상태명', '인허가일자', '폐업일자', '좌표정보(X)', '좌표정보(Y)']]
seoul_sb_df = seoul_sb_df[seoul_sb_df['사업장명'].str.contains('스타벅스')]
seoul_sb_df.reset_index(drop=True, inplace=True)
seoul_sb_df = seoul_sb_df.rename(columns={'좌표정보(X)':'X좌표값','좌표정보(Y)':'Y좌표값', '상세영업상태명' : '영업상태', '인허가일자' : '개업일자'})

In [21]:
seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str.replace('-','')
seoul_sb_df['개업일자'] = seoul_sb_df['개업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str[:6]
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].fillna('202301')
seoul_sb_df['폐업일자'] = seoul_sb_df['폐업일자'].str.replace('-','')

gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str.replace('-','')
gg_sb_df['개업일자'] = gg_sb_df['개업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str[:6]
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].fillna('202301')
gg_sb_df['폐업일자'] = gg_sb_df['폐업일자'].str.replace('-','')
gg_sb_df['영업상태'] = gg_sb_df['영업상태'].str.replace('영업/정상','영업')

In [22]:
seoul_sb_df.head()

,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,스타벅스 홍대동교점,영업,202303,202301,193414.6768,450675.0036
1,스타벅스 서울교대점,영업,201007,202301,201279.4432,442806.7092
2,스타벅스 반포자이점,영업,202201,202301,200800.4170,444989.8743
3,스타벅스 대치한티점,영업,202101,202301,204537.4607,444036.1667
4,스타벅스 대치재경빌딩B1,영업,202012,202301,205464.7204,443592.9735


In [23]:
seoul_sb_df.loc[seoul_sb_df['X좌표값'].isnull()]

,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
90,스타벅스마포이마트,영업,201201,202301,NaN,NaN
131,스타벅스 송파헬리오시티점,영업,202004,202301,NaN,NaN
136,스타벅스신용산역점,영업,201708,202301,NaN,NaN
170,스타벅스 숭례문에스지타워,영업,202008,202301,NaN,NaN
171,스타벅스 청계크리스탈스퀘어점,영업,202203,202301,NaN,NaN
178,스타벅스 서소문로점,영업,201410,202301,NaN,NaN
192,스타벅스효창공원앞역점,영업,201911,202301,NaN,NaN
204,스타벅스용산해링턴스퀘어점,영업,202012,202301,NaN,NaN
213,스타벅스 을지로2가점,영업,201605,202301,NaN,NaN
215,스타벅스 청계산입구역점,영업,201603,202301,NaN,NaN


In [24]:
gg_sb_df.head()

,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
0,스타벅스남이섬점,영업,202107,202301,246197.235727,478362.818093
1,(주)스타벅스커피코리아일산라페스타점,폐업,200308,201306,179538.773119,462032.199846
2,스타벅스 고양대로DT점,영업,202206,202301,181927.758553,463930.090165
3,스타벅스 고양덕은점,영업,202303,202301,NaN,NaN
4,스타벅스 고양원흥DT점,영업,202108,202301,188391.022888,459392.615118


In [25]:
gg_sb_df.isnull().sum()

사업장명     0
영업상태     0
개업일자     0
폐업일자     0
X좌표값    29
Y좌표값    29
dtype: int64

In [26]:
gg_sb_df.loc[gg_sb_df['X좌표값'].isnull()]

,사업장명,영업상태,개업일자,폐업일자,X좌표값,Y좌표값
3,스타벅스 고양덕은점,영업,202303,202301,NaN,NaN
23,스타벅스 지축역점,영업,202212,202301,NaN,NaN
68,스타벅스 경기광주신현DT점,영업,201812,202301,NaN,NaN
77,스타벅스 구리토평DT,영업,202112,202301,NaN,NaN
86,스타벅스 김포걸포점,영업,202203,202301,NaN,NaN
102,스타벅스 남양주묵현DT,영업,202111,202301,NaN,NaN
111,스타벅스 더북한강R,영업,202201,202301,NaN,NaN
112,스타벅스 도농역점,영업,202004,202301,NaN,NaN
206,스타벅스 광교갤러리아 9F점,영업,202002,202301,NaN,NaN
244,스타벅스커피수원트레이더스점,영업,201408,202301,NaN,NaN


In [27]:
seoul_sb_df.to_csv('./seoul_sb_처리중.csv', index=False)
gg_sb_df.to_csv('./gg_sb_처리중.csv', index=False)


## 1) X좌표, Y좌표 값을 처리해야됨